In [ ]:
%%time
import os
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
import nltk
import json
import tensorflow as tf
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.metrics import average_precision_score,roc_auc_score, roc_curve, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
np.random.seed(42)
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import ToktokTokenizer
import nltk, string, re, spacy,unicodedata, random
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


CPU times: user 11 s, sys: 2.14 s, total: 13.1 s
Wall time: 16.8 s


In [ ]:
import pandas as pd

train_df = pd.read_csv('/content/Fake_train.csv', encoding='utf-8')
test_df = pd.read_csv('/content/Fake_test_without_labels.csv', encoding='utf-8')
df2 = pd.read_csv('/content/Fake_dev.csv', encoding='utf-8')


In [ ]:
import pandas as pd

# Load the dataset from the CSV file
df = pd.read_csv('/content/Fake_train.csv')

# Displaying DataFrame in Colab
df


,text,label
0,നല്ല അവതരണം. സത്യം പുറത്തു വരട്ടെ,Fake
1,Masha Allah,Fake
2,"അന്വേഷണം കഴിയുമ്പോൾ,. C. A. A. യ്ക്ക് എതിരായ ക...",Fake
3,Illathentha avaru purath vidayittalland verenth,Fake
4,Barana pakshathin matoru niyamam.nalla moyanth...,original
...,...,...
3252,വീണാ ജോർജ് എന്ന ഒരു ആരോഗ്യ മന്ത്രി വന്നതിനു ശേ...,original
3253,ഇതാണ് പിണുവാതിര😂😂😂,original
3254,കേസ് എടുക്കണം 💯👍,original
3255,എല്ലാത്തിനേം 501 സോപ്പിട്ട് കളിപ്പിച്ച് വിടണം,original


In [ ]:
# Preprocess data
text_column = 'text'
label_column = 'label'
train_df[text_column] = train_df[text_column].fillna('')
test_df[text_column] = test_df[text_column].fillna('')

In [ ]:
# Tokenization
max_vocab_size = 10000
max_sequence_length = 100

tokenizer = Tokenizer(num_words=max_vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df[text_column])

X_train = tokenizer.texts_to_sequences(train_df[text_column])
X_test = tokenizer.texts_to_sequences(test_df[text_column])
X_train = pad_sequences(X_train, maxlen=max_sequence_length, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_sequence_length, padding='post', truncating='post')
X_train

array([[  38,  328,  111, ...,    0,    0,    0],
       [4551, 1629,    0, ...,    0,    0,    0],
       [4552, 1229, 1630, ...,    0,    0,    0],
       ...,
       [ 184, 4430,    1, ...,    0,    0,    0],
       [   1,  891,    1, ...,    0,    0,    0],
       [ 739,  127,  739, ...,    0,    0,    0]], dtype=int32)

In [ ]:
# Label Encoding
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df[label_column])
print(train_df['label'].value_counts())

label
original    1658
Fake        1599
Name: count, dtype: int64


In [ ]:
# Save to a new CSV file
train_df.to_csv('/content/Preprocessed_Fake_train.csv', index=False, encoding='utf-8')


In [ ]:
# Split train and dev data
X_train = tokenizer.texts_to_sequences(train_df[text_column])
X_dev = tokenizer.texts_to_sequences(df2[text_column])  # Assuming 'df2' is your dev dataset

# Pad sequences
X_train = pad_sequences(X_train, maxlen=max_sequence_length, padding='post', truncating='post')
X_dev = pad_sequences(X_dev, maxlen=max_sequence_length, padding='post', truncating='post')

# Label encoding for dev set
y_dev = label_encoder.transform(df2[label_column])  # Using same encoder as for the train set


In [ ]:
#logistic regrression using tfidf
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Step 1: Load your preprocessed data

# Load training, validation, and test data
train_df = pd.read_csv('/content/Fake_train.csv')
test_data = pd.read_csv('/content/Fake_test_without_labels.csv')
df2 = pd.read_csv('/content/Fake_dev.csv')

# Set the column names for text and labels
text_column = 'text'
label_column = 'label'

# Step 2: TF-IDF Vectorization
max_vocab_size = 10000
vectorizer = TfidfVectorizer(max_features=max_vocab_size)

# Vectorize the datasets
X_train = vectorizer.fit_transform(train_df[text_column])
X_dev = vectorizer.transform(df2[text_column])
X_test = vectorizer.transform(test_data[text_column])

# Step 3: Label encoding for train and dev sets
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df[label_column])
y_dev = label_encoder.transform(df2[label_column])

# Step 4: Compute class weights for balanced classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Step 5: Define and train the Logistic Regression model
model_lr = LogisticRegression(C=0.01, max_iter=1000, class_weight='balanced', random_state=42)
model_lr.fit(X_train, y_train)

# Step 6: Evaluate the model on the dev set
y_dev_pred = model_lr.predict(X_dev)

# Print evaluation metrics for the dev set
print("Validation Accuracy:", accuracy_score(y_dev, y_dev_pred))
print("Classification Report on Dev Set:")
print(classification_report(y_dev, y_dev_pred))

# Step 7: Make predictions on the test set
y_test_pred = model_lr.predict(X_test)

# Convert predictions to 'Fake' or 'Original' labels
test_labels = ['Fake' if label == 0 else 'original' for label in y_test_pred]

# Step 8: Add predictions to the test dataframe and save results
test_data['predicted_label'] = test_labels

# Save predictions to a new CSV file
output_path = '/content/logistic_regression_predicted_test.csv'
test_data.to_csv(output_path, index=False)

print(f"Predictions saved to {output_path}")

import pandas as pd
from sklearn.metrics import classification_report, accuracy_score

# Load the predicted and actual files
predicted_file = pd.read_csv('/content/logistic_regression_predicted_test.csv')  # Predicted labels
actual_file = pd.read_csv('/content/Fake_test_with_labels.csv')  # Actual labels

# Verify column names in the predicted and actual files
print("Predicted file columns:", predicted_file.columns)
print("Actual file columns:", actual_file.columns)

# Create the 'Id' column in the actual file
# Assuming the row order corresponds to IDs like Fake_01, Fake_02, ...
actual_file['Id'] = ['Fake_{:02d}'.format(i + 1) for i in range(len(actual_file))]

# Verify the 'Id' column
print("Actual file with 'Id' column:")
print(actual_file.head())

# Merge the predicted file with the actual file on the 'Id' column
merged = pd.merge(predicted_file, actual_file, on='Id')

# Extract the true labels and predicted labels from the merged DataFrame
y_true = merged['label']  # Actual labels
y_pred = merged['predicted_label']  # Predicted labels

# Compare predictions and compute evaluation metrics
print("Accuracy:", accuracy_score(y_true, y_pred))
print("Classification Report:")
print(classification_report(y_true, y_pred))

# Save the merged file with comparison for review (optional)
merged.to_csv('/content/logistic_regression_merged_comparison.csv', index=False)
print("Merged comparison file saved to /content/logistic_regression_merged_comparison.csv")


Validation Accuracy: 0.7607361963190185
Classification Report on Dev Set:
              precision    recall  f1-score   support

           0       0.79      0.71      0.75       406
           1       0.74      0.81      0.77       409

    accuracy                           0.76       815
   macro avg       0.76      0.76      0.76       815
weighted avg       0.76      0.76      0.76       815

Predictions saved to /content/logistic_regression_predicted_test.csv
Predicted file columns: Index(['Id', 'text', 'predicted_label'], dtype='object')
Actual file columns: Index(['text', 'label'], dtype='object')
Actual file with 'Id' column:
                                                text     label       Id
0  5000 ഉള്ള പോൾ  ലോഗ്‌ഡ്‌വൻ ഇപ്പോള് 250000 എന്താ...      Fake  Fake_01
1  ഓഷോ രജനീഷ്  പറഞ്ഞപോലെ  എനിക്കപ്പോൾ തോന്നിയത് അ...      Fake  Fake_02
2  ചേട്ടാ  വാർത്ത  വയ്ക്കുന്നത്  കേരളത്തിലാണ്  സം...      Fake  Fake_03
3             Shame for entire Woman&#39;s of Kerala  original  Fake_

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Step 1: Load your preprocessed data

# Load training, validation, and test data
train_df = pd.read_csv('/content/Fake_train.csv')
test_data = pd.read_csv('/content/Fake_test_without_labels.csv')
df2 = pd.read_csv('/content/Fake_dev.csv')

# Set the column names for text and labels
text_column = 'text'
label_column = 'label'

# Step 2: TF-IDF Vectorization
max_vocab_size = 10000
vectorizer = TfidfVectorizer(max_features=max_vocab_size)

# Vectorize the datasets
X_train = vectorizer.fit_transform(train_df[text_column])
X_dev = vectorizer.transform(df2[text_column])
X_test = vectorizer.transform(test_data[text_column])

# Step 3: Label encoding for train and dev sets
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df[label_column])
y_dev = label_encoder.transform(df2[label_column])

# Step 4: Compute class weights for balanced classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Step 5: Define and train the Decision Tree model
model_dt = DecisionTreeClassifier(max_depth=10, class_weight='balanced', random_state=42)
model_dt.fit(X_train, y_train)

# Step 6: Evaluate the model on the dev set
y_dev_pred = model_dt.predict(X_dev)

# Print evaluation metrics for the dev set
print("Validation Accuracy:", accuracy_score(y_dev, y_dev_pred))
print("Classification Report on Dev Set:")
print(classification_report(y_dev, y_dev_pred))

# Step 7: Make predictions on the test set
y_test_pred = model_dt.predict(X_test)

# Convert predictions to 'Fake' or 'Original' labels
test_labels = ['Fake' if label == 0 else 'original' for label in y_test_pred]

# Step 8: Add predictions to the test dataframe and save results
test_data['predicted_label'] = test_labels

# Save predictions to a new CSV file
output_path = '/content/decision_tree_predicted_test.csv'
test_data.to_csv(output_path, index=False)

print(f"Predictions saved to {output_path}")

# Merging to compare with actual labels

# Load the predicted and actual files
predicted_file = pd.read_csv('/content/decision_tree_predicted_test.csv')
actual_file = pd.read_csv('/content/Fake_test_with_labels.csv')

# Verify column names in the predicted and actual files
print("Predicted file columns:", predicted_file.columns)
print("Actual file columns:", actual_file.columns)

# Create the 'Id' column in the actual file
actual_file['Id'] = ['Fake_{:02d}'.format(i + 1) for i in range(len(actual_file))]

# Merge the predicted file with the actual file on the 'Id' column
merged = pd.merge(predicted_file, actual_file, on='Id')

# Extract the true labels and predicted labels from the merged DataFrame
y_true = merged['label']
y_pred = merged['predicted_label']

# Compare predictions and compute evaluation metrics
print("Accuracy:", accuracy_score(y_true, y_pred))
print("Classification Report:")
print(classification_report(y_true, y_pred))

# Save the merged file with comparison for review (optional)
merged.to_csv('/content/decision_tree_merged_comparison.csv', index=False)
print("Merged comparison file saved to /content/decision_tree_merged_comparison.csv")


Validation Accuracy: 0.6466257668711657
Classification Report on Dev Set:
              precision    recall  f1-score   support

           0       0.88      0.33      0.49       406
           1       0.59      0.96      0.73       409

    accuracy                           0.65       815
   macro avg       0.74      0.65      0.61       815
weighted avg       0.74      0.65      0.61       815

Predictions saved to /content/decision_tree_predicted_test.csv
Predicted file columns: Index(['Id', 'text', 'predicted_label'], dtype='object')
Actual file columns: Index(['text', 'label'], dtype='object')
Accuracy: 0.6300294406280668
Classification Report:
              precision    recall  f1-score   support

        Fake       0.86      0.31      0.45       507
    original       0.58      0.95      0.72       512

    accuracy                           0.63      1019
   macro avg       0.72      0.63      0.59      1019
weighted avg       0.72      0.63      0.59      1019

Merged compari

In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Step 1: Load your preprocessed data
train_df = pd.read_csv('/content/Fake_train.csv')
test_data = pd.read_csv('/content/Fake_test_without_labels.csv')
df2 = pd.read_csv('/content/Fake_dev.csv')

# Set the column names for text and labels
text_column = 'text'
label_column = 'label'

# Preprocess the data using TF-IDF Vectorization
max_vocab_size = 10000

# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=max_vocab_size)

# Fit and transform the train data, transform dev and test data
X_train = vectorizer.fit_transform(train_df[text_column]).toarray()
X_dev = vectorizer.transform(df2[text_column]).toarray()
X_test = vectorizer.transform(test_data[text_column]).toarray()

# Label encoding for train and dev sets
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_df[label_column])
y_dev = label_encoder.transform(df2[label_column])

# Step 2: Compute class weights for balanced classes
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Step 3: Define and train the SVM model
model_svm = SVC(class_weight='balanced', kernel='linear', random_state=42)
model_svm.fit(X_train, y_train)

# Step 4: Evaluate the model on the dev set
y_dev_pred = model_svm.predict(X_dev)

# Print evaluation metrics for the dev set
print("Validation Accuracy:", accuracy_score(y_dev, y_dev_pred))
print("Classification Report on Dev Set:")
print(classification_report(y_dev, y_dev_pred))

# Step 5: Make predictions on the test set
y_test_pred = model_svm.predict(X_test)

# Convert predictions to 'Fake' or 'Original' labels
test_labels = ['Fake' if label == 0 else 'original' for label in y_test_pred]

# Step 6: Add predictions to the test dataframe and save results
test_data['predicted_label'] = test_labels

# Save predictions to a new CSV file
output_path = '/content/svm_predicted_test.csv'
test_data.to_csv(output_path, index=False)

print(f"Predictions saved to {output_path}")

# Merging to compare with actual labels
# Load the predicted and actual files
predicted_file = pd.read_csv('/content/svm_predicted_test.csv')  # Predicted labels
actual_file = pd.read_csv('/content/Fake_test_with_labels.csv')  # Actual labels

# Verify column names in the predicted and actual files
print("Predicted file columns:", predicted_file.columns)
print("Actual file columns:", actual_file.columns)

# Create the 'Id' column in the actual file
# Assuming the row order corresponds to IDs like Fake_01, Fake_02, ...
actual_file['Id'] = ['Fake_{:02d}'.format(i + 1) for i in range(len(actual_file))]

# Verify the 'Id' column
print("Actual file with 'Id' column:")
print(actual_file.head())

# Merge the predicted file with the actual file on the 'Id' column
merged = pd.merge(predicted_file, actual_file, on='Id')

# Extract the true labels and predicted labels from the merged DataFrame
y_true = merged['label']  # Actual labels
y_pred = merged['predicted_label']  # Predicted labels

# Compare predictions and compute evaluation metrics
print("Accuracy:", accuracy_score(y_true, y_pred))
print("Classification Report:")
print(classification_report(y_true, y_pred))

# Save the merged file with comparison for review (optional)
merged.to_csv('/content/svm_merged_comparison.csv', index=False)
print("Merged comparison file saved to /content/svm_merged_comparison.csv")


Validation Accuracy: 0.7815950920245399
Classification Report on Dev Set:
              precision    recall  f1-score   support

           0       0.79      0.76      0.78       406
           1       0.77      0.80      0.79       409

    accuracy                           0.78       815
   macro avg       0.78      0.78      0.78       815
weighted avg       0.78      0.78      0.78       815

Predictions saved to /content/svm_predicted_test.csv
Predicted file columns: Index(['Id', 'text', 'predicted_label'], dtype='object')
Actual file columns: Index(['text', 'label'], dtype='object')
Actual file with 'Id' column:
                                                text     label       Id
0  5000 ഉള്ള പോൾ  ലോഗ്‌ഡ്‌വൻ ഇപ്പോള് 250000 എന്താ...      Fake  Fake_01
1  ഓഷോ രജനീഷ്  പറഞ്ഞപോലെ  എനിക്കപ്പോൾ തോന്നിയത് അ...      Fake  Fake_02
2  ചേട്ടാ  വാർത്ത  വയ്ക്കുന്നത്  കേരളത്തിലാണ്  സം...      Fake  Fake_03
3             Shame for entire Woman&#39;s of Kerala  original  Fake_04
4  135 code j